In [1]:
import os
import cv2
import numpy as np

In [2]:
def load_BreaKHis400X_data(train_dir, test_dir):
    label_to_index = {'benign':0, 'malignant':1}

    def load_images_from_directory(directory):
        images = []
        labels = []
        for label_name in os.listdir(directory):
            label_index = label_to_index[label_name]
            label_dir = os.path.join(directory, label_name)
            for image_file in os.listdir(label_dir):
                image_path = os.path.join(label_dir, image_file)
                image = cv2.imread(image_path)
                image = cv2.resize(image, (32, 32))
                images.append(image)
                labels.append(label_index)
        return np.array(images), np.array(labels)
    
    x_train, y_train = load_images_from_directory(train_dir)
    x_test, y_test = load_images_from_directory(test_dir)

    return x_train, y_train, x_test, y_test



In [3]:
train_directory = "/kaggle/input/breakhis-400x/BreaKHis 400X/train"
test_directory = "/kaggle/input/breakhis-400x/BreaKHis 400X/test"



In [4]:
x_train, y_train, x_test, y_test = load_BreaKHis400X_data(train_directory, test_directory)

In [5]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

In [6]:
permutation = np.random.permutation(len(x_train))

In [7]:
x_train_shuffled = x_train[permutation]
y_train_shuffled = y_train[permutation]

In [8]:
permutation_test = np.random.permutation(len(x_test))

In [9]:
x_test_shuffled = x_test[permutation_test]
y_test_shuffled = y_test[permutation_test]

In [10]:
import tensorflow as tf
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.models import Sequential
from keras.optimizers import Adam, AdamW, Adamax, Lion
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

2024-05-21 21:35:35.826142: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 21:35:35.826298: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 21:35:35.985970: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [20]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)
train_datagen.fit(x_train)

In [21]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.layers import LeakyReLU
model = Sequential([
    
    Flatten(input_shape=(32, 32, 3)),
    Dense(512, activation='leaky_relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='leaky_relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='leaky_relu'),
    BatchNormalization(),
    Dense(2, activation='sigmoid')
    
])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [22]:
optimizer = Adam(learning_rate=0.001)

In [23]:
model.compile(optimizer=optimizer,   loss='sparse_categorical_crossentropy', metrics=['accuracy']) 



In [15]:
#early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

In [17]:
#model.fit(x_train, y_train, batch_size=15,epochs=25)
#model.fit(x_train, y_train, batch_size=15, epochs=25, validation_data=(x_test, y_test), callbacks=[early_stopping, reduce_lr])

In [24]:
model.fit(train_datagen.flow(x_train, y_train, batch_size=15),epochs=25)

Epoch 1/25
77/77 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.6781 - loss: 0.8144
Epoch 2/25
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.7815 - loss: 0.5154
Epoch 3/25
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8089 - loss: 0.4715
Epoch 4/25
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.8165 - loss: 0.4443
Epoch 5/25
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.7976 - loss: 0.4946
Epoch 6/25
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.8185 - loss: 0.4263
Epoch 7/25
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.8252 - loss: 0.4236
Epoch 8/25
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8092 - loss: 0.4183
Epoch 9/25
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.8043 - loss: 0.4472
Epoch 10/25
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8229 - loss: 0.4314
Epoch 11/25
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.8064 - loss: 0.4360
Epoch 12/25
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy:

In [25]:
results = model.evaluate(x_test,  y_test, verbose = 0) 
print('test loss, test acc:', results)

test loss, test acc: [0.39425602555274963, 0.8385320901870728]
